In [ ]:
#Build an Artificial Neural Network by implementing the Back-propagation algorithm and test the same using appropriate data sets.

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 1. Load and preprocess the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, parser='auto')  
X = mnist.data.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
y = OneHotEncoder(sparse_output=False).fit_transform(mnist.target.to_numpy().reshape(-1, 1))  # Encode target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Define the Neural Network Architecture
input_size = X_train.shape[1]  # 784 input neurons (28x28 images)
hidden_size = 128  # Number of hidden neurons
output_size = y_train.shape[1]  # 10 output classes
learning_rate = 0.01

# Initialize weights and biases
np.random.seed(42)
weights_input_hidden = np.random.randn(input_size, hidden_size) * 0.01
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size) * 0.01
bias_output = np.zeros((1, output_size))

# Activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 3. Train the Network
epochs = 10
batch_size = 64
num_batches = X_train.shape[0] // batch_size

for epoch in range(epochs):
    for batch in range(num_batches):
        # Get batch data
        start = batch * batch_size
        end = start + batch_size
        X_batch = X_train[start:end]
        y_batch = y_train[start:end]
        
        # Forward Propagation
        hidden_input = np.dot(X_batch, weights_input_hidden) + bias_hidden
        hidden_output = sigmoid(hidden_input)
        final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
        final_output = sigmoid(final_input)
        
        # Calculate the error
        error = y_batch - final_output
        
        # Back-propagation
        d_output = error * sigmoid_derivative(final_output)
        error_hidden_layer = d_output.dot(weights_hidden_output.T)
        d_hidden = error_hidden_layer * sigmoid_derivative(hidden_output)
        
        # Update weights and biases
        weights_hidden_output += hidden_output.T.dot(d_output) * learning_rate
        bias_output += np.sum(d_output, axis=0, keepdims=True) * learning_rate
        weights_input_hidden += X_batch.T.dot(d_hidden) * learning_rate
        bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * learning_rate

    # Print progress
    hidden_input = np.dot(X_train, weights_input_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)
    final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
    final_output = sigmoid(final_input)
    train_loss = np.mean(np.square(y_train - final_output))
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}')

# 4. Evaluate the Network
hidden_input = np.dot(X_test, weights_input_hidden) + bias_hidden
hidden_output = sigmoid(hidden_input)
final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
final_output = sigmoid(final_input)

# Convert predictions to class labels
predicted_classes = np.argmax(final_output, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_classes == true_classes) * 100
print(f'Test Accuracy: {accuracy:.2f}%')


Epoch 1/10, Loss: 0.0333
Epoch 2/10, Loss: 0.0202
Epoch 3/10, Loss: 0.0168
Epoch 4/10, Loss: 0.0149
Epoch 5/10, Loss: 0.0136
Epoch 6/10, Loss: 0.0126
Epoch 7/10, Loss: 0.0118
Epoch 8/10, Loss: 0.0111
Epoch 9/10, Loss: 0.0105
Epoch 10/10, Loss: 0.0100
Test Accuracy: 93.93%
